# **Alineamiento múltiple de secuencias** 
## Software utilizado: BLAST y MAFFT

Inicialmente importamos las librerías necesarias para ejecutar con éxito el alineamiento múltiple

Posteriormente buscamos datos como secuencia y nombre de GLUT1 en base al código de indexación en la base de datos UNIPROT y lo guardamos en la carpeta *"../run"*

In [1]:
# Libraries import
from Bio import ExPASy, SwissProt
from Bio import SeqIO
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqUtils import GC123
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.PDB import PDBList
import os
import xml.etree.ElementTree as ET
from io import StringIO
import subprocess
import pandas as pd
import requests

In [10]:
uniprot_code = 'P11166' # GLUT1 - SLC2A1

#=== Fetch the sequence from UniProt ===
handle = ExPASy.get_sprot_raw(uniprot_code)
record = SeqIO.read(handle, 'swiss')
prot_nm = record.name
prot_sq = record.seq

#=== Save as FASTA file ===
output_dir = "../run/blast_mafft"
os.makedirs(output_dir, exist_ok=True)
fasta_path = os.path.join(output_dir, f"{prot_nm}.fasta")

with open(fasta_path, 'w') as fasta_file:
    SeqIO.write(record, fasta_file, 'fasta')

#=== Print the sequence ===
print(f'Nombre de la proteína: ', prot_nm)
print(f"Archivo FASTA guardado en: {fasta_path}")

Nombre de la proteína:  GTR1_HUMAN
Archivo FASTA guardado en: ../run/blast_mafft/GTR1_HUMAN.fasta


Si conoces el PDB_ID de la o las proteinas que quieres alinear, ejecuta esta celda.

Si no es así, ejecuta la celda siguiente

In [12]:
def download_fasta_from_pdb(pdb_ids, outputdir):
    """
    Descarga secuencias FASTA para una lista de IDs de PDB dados.
    """
    base_url = "https://www.uniprot.org/uploadlists/"
    
    print("Descargando archivos FASTA...")
    
    for pdb_id in pdb_ids:
        fasta_url = f"https://www.rcsb.org/fasta/entry/{pdb_id}/download"
        os.makedirs(output_dir, exist_ok=True)

        try:
            response = requests.get(fasta_url)
            response.raise_for_status()  # Genera una excepción para errores HTTP (4xx o 5xx)
            
            # El contenido de la respuesta es el archivo FASTA
            fasta_content = response.text
            
            # Guarda el contenido en un archivo FASTA
            file_name = f"{pdb_id}.fasta"
            fullpath = os.path.join(output_dir, file_name)
            
            with open(fullpath, 'w') as f:
                f.write(fasta_content)
            
            print(f"✔️ {file_name} descargado exitosamente.")
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Error al descargar {pdb_id}: {e}")
        except Exception as e:
            print(f"❌ Ocurrió un error inesperado con {pdb_id}: {e}")

# --- Uso del código ---
if __name__ == "__main__":
    # Puedes cambiar esta lista por los IDs de PDB que necesites
    pdb_identifiers = ['4pyp', '5eqg', '5eqh', '5eqi', '6tha'] 
    output_dir = "../run/comparativa"

    download_fasta_from_pdb(pdb_identifiers, output_dir)

Descargando archivos FASTA...
✔️ 4pyp.fasta descargado exitosamente.
✔️ 5eqg.fasta descargado exitosamente.
✔️ 5eqh.fasta descargado exitosamente.
✔️ 5eqi.fasta descargado exitosamente.
✔️ 6tha.fasta descargado exitosamente.


Una vez obtenidos esos datos, creamos una subcarpeta para guardar nuestros alineamientos.
El fin de realizar este paso es identificar posibles homólogos de las secuencias alineadas en base a nuestra proteína, lo que puede proporcionar información valiosa sobre su función, origen evolutivo y relaciones funcionales. 

Al ejecutar BLAST, se genera un archivo *.xml*, el cual guardaremos en la carpeta recíen creada

In [13]:
# === General settings ===
output_dir = "../run/blast_mafft"
os.makedirs(output_dir, exist_ok=True)

xml_path = os.path.join(output_dir, "blast_results.xml")
fasta_path = os.path.join(output_dir, "hits_blast.fasta")
aligned_path = os.path.join(output_dir, "alignment_mafft.fasta")

#=== Run BLASTp online ===
print("Ejecutando BLASTp...")
try:
    result = NCBIWWW.qblast('blastp', 'pdb', prot_sq, hitlist_size=10)
    with open(xml_path, 'w') as out_xml:
        out_xml.write(result.read())
    result.close()
    print(f'Resultado BLAST guardado en {xml_path}')
except Exception as ex:
    print("Error durante BLAST:", ex)
    exit()

Ejecutando BLASTp...


KeyboardInterrupt: 

Luego, de los datos obtenidos, los analizaremos para facilitar la redacción y extracción de información importante como lo es la secuencia de cada proteína obtenida en el proceso anterior

In [14]:
#=== Parse results and extract sequences ===
print("Extrayendo secuencias...")
records = []
with open(xml_path) as result_file:
    blast_record = NCBIXML.read(result_file)

i = 1
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.sbjct and isinstance(hsp.sbjct, str):
            clean_seq = hsp.sbjct.replace("-", "").strip()
            seq = Seq(clean_seq)
            rec_id = f'{alignment.accession}_{i}'
            description = alignment.hit_def
            record = SeqRecord(seq, id=rec_id, description=description)
            records.append(record)
            i += 1
        break
    if i > 10:
        break

if not records: # Sequence validation
    print("No se extrajeron secuencias válidas")
    exit()

SeqIO.write(records, fasta_path, "fasta") # Save in FASTA format
print(f"{len(records)} secuencias guardadas en: {fasta_path}")


Extrayendo secuencias...
10 secuencias guardadas en: ../run/blast_mafft/hits_blast.fasta


A continuación se ejecutará el alineamiento multiple de secuencias con MAFFT. Dentro del código se crean directorios para alojar los outputs (.aln y .fasta) generados del alineamiento y se genera una tabla para visualizar el alineamiento y confirmar su correcta ejecución 

In [2]:
def run_mafft(fasta_path: str, output_dir: str, base_name: str = "MSA"):
    print("Ejecutando alineamiento múltiple con MAFFT")

    os.makedirs(output_dir, exist_ok=True) # confirm directory

    try:
        result = subprocess.run(
            ["mafft", "--auto", fasta_path],
            capture_output=True,
            text=True,
            check=True
        )
    except subprocess.CalledProcessError as er:
        print("Error al ejecutar MAFFT:")
        print(er.stderr)
        return None
    
    # Parsing alignment
    alignment = AlignIO.read(StringIO(result.stdout), 'fasta')

    # Save formats
    fasta_out = os.path.join(output_dir, f'{base_name}.fasta')
    clustal_out = os.path.join(output_dir, f'{base_name}.aln')

    AlignIO.write(alignment, fasta_out, "fasta")
    AlignIO.write(alignment, clustal_out, "clustal")

    print(f'\n Alineamiento guardado en:')
    print(f'    * FASTA     : {fasta_out}')
    print(f'    * CLUSTAL   : {clustal_out}')

    return alignment # for visualize

#=== Running alignment ===
output_dir = '../run/blast_mafft'
fasta_path = '../run/blast_mafft/hits_blast.fasta'

alignment = run_mafft(fasta_path, output_dir)       # For execution


#=== Visualize the alignment as a table===
if alignment:
    print("\n Vista tabular del alineamiento")
    df = pd.DataFrame({
        record.id: list(str(record.seq)) for record in alignment
    }).T        # Show each sequence in rows
    df.columns = [f'Pos {i+1}'for i in range(df.shape[1])]
    display(df)

Ejecutando alineamiento múltiple con MAFFT

 Alineamiento guardado en:
    * FASTA     : ../run/blast_mafft/MSA.fasta
    * CLUSTAL   : ../run/blast_mafft/MSA.aln

 Vista tabular del alineamiento


,Pos 1,Pos 2,Pos 3,Pos 4,Pos 5,Pos 6,Pos 7,Pos 8,Pos 9,Pos 10,...,Pos 496,Pos 497,Pos 498,Pos 499,Pos 500,Pos 501,Pos 502,Pos 503,Pos 504,Pos 505
5EQG_A_1,M,E,P,S,S,K,-,-,-,-,...,F,H,P,L,G,A,D,S,Q,V
6THA_A_2,M,E,P,S,S,K,-,-,-,-,...,F,H,P,L,G,A,D,S,Q,V
4PYP_A_3,M,E,P,S,S,K,-,-,-,-,...,F,H,P,L,G,A,D,S,Q,V
7SPS_A_4,-,-,-,-,T,Q,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4ZW9_A_5,-,-,-,-,T,Q,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5C65_A_6,-,-,-,-,T,Q,-,-,-,-,...,F,Q,-,-,-,-,-,-,-,-
7SPT_A_7,-,-,-,-,T,Q,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7WSM_A_8,-,E,P,P,Q,Q,-,-,-,-,...,L,E,Y,L,G,P,D,E,N,-
4YBQ_A_9,M,E,K,E,D,Q,-,E,K,T,...,-,-,-,-,-,-,-,-,-,-
4YB9_D_10,M,E,P,Q,D,P,V,K,R,E,...,-,-,-,-,-,-,-,-,-,-


In [3]:
alignmentfile = '../run/blast_mafft/MSA.fasta'
formato = 'fasta'
conservation_umbral = 0.95

# Cargar el alineamiento múltiple desde un archivo
alineamiento = AlignIO.read(f'{alignmentfile}', "fasta")
# Inicializar un diccionario para almacenar la conservación
conservacion = {}
# Calcular la conservación en cada posición
for i in range(alineamiento.get_alignment_length()):
    columna = alineamiento[:, i]
    # Contar la frecuencia de cada residuo en la columna
    frecuencias = {}
    for residuo in columna:
        if residuo not in frecuencias:
            frecuencias[residuo] = 0
        frecuencias[residuo] += 1
    
    # Determinar el residuo más frecuente
    residuo_mas_frecuente = max(frecuencias, key=frecuencias.get)
    conservacion[i] = (residuo_mas_frecuente, frecuencias[residuo_mas_frecuente])
# Imprimir los resultados
for posicion, (residuo, frecuencia) in conservacion.items():
    print(f"Posición {posicion + 1}: Residuo {residuo} con frecuencia {frecuencia}")

Posición 1: Residuo M con frecuencia 5
Posición 2: Residuo E con frecuencia 6
Posición 3: Residuo P con frecuencia 5
Posición 4: Residuo - con frecuencia 4
Posición 5: Residuo T con frecuencia 4
Posición 6: Residuo Q con frecuencia 6
Posición 7: Residuo - con frecuencia 9
Posición 8: Residuo - con frecuencia 8
Posición 9: Residuo - con frecuencia 8
Posición 10: Residuo - con frecuencia 8
Posición 11: Residuo - con frecuencia 8
Posición 12: Residuo K con frecuencia 8
Posición 13: Residuo L con frecuencia 5
Posición 14: Residuo T con frecuencia 10
Posición 15: Residuo P con frecuencia 5
Posición 16: Residuo A con frecuencia 4
Posición 17: Residuo L con frecuencia 9
Posición 18: Residuo I con frecuencia 4
Posición 19: Residuo L con frecuencia 6
Posición 20: Residuo A con frecuencia 10
Posición 21: Residuo V con frecuencia 4
Posición 22: Residuo T con frecuencia 4
Posición 23: Residuo V con frecuencia 4
Posición 24: Residuo A con frecuencia 10
Posición 25: Residuo V con frecuencia 4
Posici